In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import json
import os
import sys
from ampel.contrib.hu.util import AmpelHealpix

In [24]:
map_dir = (
    "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817"
)
map_name = "gw170817.fits.gz"
ah = AmpelHealpix.AmpelHealpix(map_name=map_name, save_dir=map_dir)
ah.process_map()
trigger_time = ah.trigger_time
print(trigger_time)

2457983.0285234866


In [25]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable/TransientTable_gw170817.csv"
# filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable_gw170817.json"

dataframe = pd.read_csv(filename, sep=";")

dataframe.head()

,Unnamed: 0,ampel_z,ampel_z_precision,ampel_dist,possis_abspeak,possis_chisq,possis_ndof,chisquares,grb_spatial,kilonovaness,...,map_dist,map_dist_unc,map_name,trigger_time,map_seed,stock,ztf_id,ra,dec,channels
0,6,0.00968,0.003,9.947793,-15.042757,56.679054,5,"{'bns_maurice_1': {'chisq': 447.5006648962226,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
1,7,0.00968,0.003,9.947793,-15.045842,67.653863,6,"{'bns_maurice_1': {'chisq': 448.875205475847, ...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
2,5,0.00968,0.003,9.947793,-15.043340,56.525346,4,{'bns_maurice_1': {'chisq': 402.80623969763207...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
3,14,0.00968,0.003,9.947793,-15.043978,411.993251,13,"{'bns_maurice_1': {'chisq': 643.6967885978046,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
4,12,0.00968,0.003,9.947793,-15.013035,356.732751,11,"{'bns_maurice_1': {'chisq': 617.4200420093252,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817


In [26]:
def jsonify(string: str):
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [27]:
risedec_df = pd.DataFrame()

pos_rowslist_dict = {
    "bns_maurice_1": [],
    "bns_maurice_2": [],
    "bns_maurice_3": [],
    "bns_maurice_4": [],
}

rowslist = []
for k, row in dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    # print(tmp_str)
    dec_dict = json.loads(dec_str)
    # print(dec_dict)

    rowslist.append(dec_dict)

    pos_str = jsonify(row["chisquares"])
    pos_dict = json.loads(pos_str)
    # print(pos_dict)
    for key, value in pos_dict.items():
        value["ndet"] = dec_dict["ndet"]
        value["latest_det"] = dec_dict["jd_last"]
        value["first_det"] = dec_dict["jd_det"]
        value["kilonovaness"] = row["kilonovaness"]
        pos_rowslist_dict[key].append(value)
    # print(row["rise_decline"]["ndet"])

risedec_df = pd.DataFrame(rowslist)

pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="latest_det")

In [28]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(
        go.Scatter(
            x=pos_df_dict[key]["latest_det"] - trigger_time,
            y=pos_df_dict[key]["kilonovaness"],
            name=key,
            mode="markers",
        )
    )

fig.update_layout(
    {"xaxis_title": "days from first detection", "yaxis_title": "kilonovaness"}
)
fig.show()

In [29]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(
        go.Scatter(
            x=pos_df_dict[key]["ndet"],
            y=pos_df_dict[key]["kilonovaness"],
            name=key,
            mode="markers",
        )
    )

fig.update_layout(
    {"xaxis_title": "Detections", "yaxis_title": "kilonovaness"}
)
fig.show()

In [30]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(
        go.Scatter(
            x=pos_df_dict[key]["latest_det"] - trigger_time,
            y=pos_df_dict[key]["chisq"] / pos_df_dict[key]["ndof"],
            name=key,
            mode="markers",
        )
    )

fig.update_layout(
    {"xaxis_title": "days from first detection", "yaxis_title": "Reduced chisqu"}
)
fig.show()

In [31]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(
        go.Scatter(
            x=pos_df_dict[key]["latest_det"] - trigger_time,
            y=pos_df_dict[key]["obspeak_model_B"],
            name=key,
        )
    )

fig.update_layout(
    {"xaxis_title": "days from first detection", "yaxis_title": "Obspeak"}
)
fig.show()

In [32]:
lightcurve_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/lightcurve.csv"
lightcurve_df = pd.read_csv(lightcurve_file, sep=";", comment="#")
lightcurve_df.dropna(how="all", inplace=True)
lightcurve_df.tail()

,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JD,Phase,Filter,mobs,mobs_sigma,Unnamed: 5,Instrument,upper_limit,UT_time
149,149,149,149,149,149,149,149,149,149,119,2.457997e+06,13.50,j,NaN,NaN,21.16,NaN,True,NaN
150,150,150,150,150,150,150,150,150,150,130,2.457998e+06,15.47,h,NaN,NaN,20.50,NaN,True,NaN
151,151,151,151,151,151,151,151,151,151,138,2.457998e+06,15.47,ks,20.23,-0.10,NaN,NaN,NaN,NaN
152,152,152,152,152,152,152,152,152,152,139,2.458001e+06,18.46,ks,20.81,-0.13,NaN,NaN,NaN,NaN
153,153,153,153,153,153,153,153,153,153,140,2.458001e+06,18.49,ks,20.93,-0.17,NaN,NaN,NaN,NaN


In [33]:
lightcurve_df["UT_time"] = pd.to_datetime(lightcurve_df["UT_time"])

In [34]:
tmp_lc_df = lightcurve_df[lightcurve_df["JD"].isna()].copy()
lightcurve_df["Filter"] = lightcurve_df["Filter"].str.lower()

tmp_lc_df["JD"] = pd.DatetimeIndex(tmp_lc_df["UT_time"]).to_julian_date()

lightcurve_df.update(tmp_lc_df)

lightcurve_df["JD"] = lightcurve_df["JD"].astype(float)
lightcurve_df = lightcurve_df.sort_values(by="JD", ascending=True)

lightcurve_df.to_csv(lightcurve_file, sep=";")

In [35]:
lightcurve_df["JD"][134:147]

134    2.457991e+06
135    2.457992e+06
136    2.457992e+06
137    2.457992e+06
138    2.457993e+06
139    2.457994e+06
142    2.457995e+06
140    2.457995e+06
141    2.457995e+06
143    2.457995e+06
144    2.457995e+06
145    2.457996e+06
146    2.457996e+06
Name: JD, dtype: float64

In [36]:
filter_bands = lightcurve_df["Filter"].str.lower().unique()
filter_bands = ["r", "g", "i"]
fig = go.Figure()

for filter in filter_bands:
    filter_df = lightcurve_df[lightcurve_df["Filter"] == filter].copy()
    fig.add_trace(
        go.Scatter(
            x=filter_df["JD"] - trigger_time,
            y=filter_df["mobs"],
            name=filter,
            error_y_array=filter_df["mobs_sigma"],
        )
    )

fig.update_layout({"yaxis_autorange": "reversed"})
fig.show()

In [37]:
import plotly.graph_objects as go
from scipy.optimize import curve_fit


def f(x, a, b):
    return a * x + b


def flux(mag):
    flux = np.power(10, (25 - mag) / 2.5)
    return flux


def fluxerr(magerr, flux):
    fluxerr = np.log(10) * magerr / 2.5 * flux
    return fluxerr

In [38]:
from scipy.special import erfinv

In [39]:
filter = filter_bands[0]
colors = {"r": "red", "g": "green", "i": "purple"}
fig = go.Figure()
fig2 = go.Figure()
slopes_sigs = []
for k, filter in enumerate(filter_bands[:]):
    print(filter)
    tmp_df = (
        lightcurve_df[lightcurve_df["Filter"] == filter].copy().dropna(subset="mobs")
    )
    datapoints = flux(tmp_df["mobs"].values)

    errors = fluxerr(tmp_df["mobs_sigma"].values, datapoints)
    times = tmp_df["JD"].values - trigger_time

    max_time_ind = sum(times <= 5)
    # print(max_ind, max_time_ind)
    # print(datapoints, errors)

    max_ind = np.argmax(datapoints)

    result = curve_fit(
        f=f,
        xdata=times[max_ind:max_time_ind],
        ydata=datapoints[max_ind:max_time_ind],
        sigma=errors[max_ind:max_time_ind],
    )
    fitted = f(times[max_ind:max_time_ind], *result[0])

    residuals = np.sort(fitted - datapoints[max_ind:max_time_ind])
    N = len(residuals)
    print(residuals)
    cum_prob = np.array([i / (N + 1) for i in range(N)])
    quantiles = np.sqrt(2) * erfinv(2 * cum_prob - 1)
    fig2.add_trace(
        go.Scatter(
            x=quantiles,
            y=residuals,
            mode="markers",
            name=filter,
            line_color=colors[filter],
        )
    )

    slopes_sigs.append(result[0][0] / np.sqrt(result[1][0][0]))

    fig.add_trace(
        go.Scatter(
            x=times,
            y=datapoints,
            name=filter,
            error_y_array=errors,
            line_color=colors[filter],
        )
    )
    fig.add_trace(
        go.Scatter(
            x=times[max_ind:max_time_ind],
            y=fitted,
            mode="lines",
            line={"dash": "dash"},
            name=f"fit {filter}",
            line_color=colors[filter],
        )
    )
    fig.add_vline(
        x=times[max_ind], line_color="grey"
    )  # , annotation={"text": "Maximum"}, annotation_position= "left")
fig.update_layout(
    {  # "yaxis_autorange": "reversed",
        "width": 900,
        "xaxis_title": "Time from merger (days)",
        "yaxis_title": "Observed magnitude (AB)",
        "title": "Rise/Decline check for AT2017gfo",
    }
)
fig2.update_layout({"width": 900})
fig.show()
fig2.show()
print(slopes_sigs, sum(slopes_sigs))
# fig.write_image("./pics/random/lightcurve.pdf")

r
[-627.1932226  -215.8152784  -182.91618381 -135.28683121  -37.3143836
  -17.82275252  -14.08938615   34.06730497   48.8045348    50.28702149
   77.37682523   80.44265488]
g
[-761.13543073 -725.85214651 -142.85613124  -94.71007243  -79.02601428
   -7.01949489   -6.3323624    47.8428374    64.77227424   80.97327119]
i
[-520.56799175 -149.76164165  -94.02694445  -84.07999782  -68.13705088
  -39.41581742  -21.07121351   52.68743438   57.72676335   78.36609211]


[-5.596939672466502, -3.268912905081542, -11.190193234848724] -20.056045812396768


In [44]:
dataframe[["kilonovaness", "gaus_percent", "ndet"]].sort_values(by="ndet")

,kilonovaness,gaus_percent,ndet
39,12,1.230474,1
38,12,1.230474,2
37,16,1.230474,3
35,16,1.230474,4
36,16,1.230474,5
2,19,1.230474,6
0,19,1.230474,7
1,19,1.230474,8
7,19,1.230474,8
6,19,1.230474,9
